In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [2]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="montuno",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

## Add Documents

In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader

loader = TextLoader("../data/real-state/proyectos.md")  # Reemplaza con la ruta de tu archivo
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # Tamaño de cada chunk (en caracteres)
    chunk_overlap=20,  # Superposición entre chunks
    length_function=len,  # Función para calcular la longitud del texto
)
chunks = text_splitter.split_documents(documents)

# 4. Almacenar los embeddings en una base de datos vectorial (FAISS en este caso)
vector_db = vector_store.from_documents(chunks, embeddings)


## Query From vector store

In [4]:
results = vector_store.similarity_search(
    "cash",
    k=2,
    filter={"source": "news"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [5]:
docs = vector_store.similarity_search_with_score("how many cash stolen?",k=2)
print(docs)

[]


## Retriever

In [6]:
retriever = vector_store.as_retriever()

## Prompt template

## RAG Chain

In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

model = "gpt-3.5-turbo"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
template = """Responde las siguientes preguntas con base en los siguientes contexto:
Eres un asistente virtual especializado en una agencia inmobiliaria. Tu tarea es proporcionar información precisa y útil sobre los proyectos inmobiliarios disponibles. Utiliza únicamente la información proporcionada en los documentos para responder, el año actual es 2025.

Proyectos disponibles:
1. **Proyecto A**: Residencial "Bosque Verde"
2. **Proyecto B**: Comercial "Plaza Sol"
3. **Proyecto C**: Residencial de lujo "Altos del Mar"

Instrucciones:
- Responde de manera clara y concisa.
- Si no tienes información suficiente, indica que no cuentas con los detalles necesarios.
- Prioriza la información sobre ubicación, precios, características y disponibilidad.

{context}
"""
prompt_template = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [13]:
response = rag_chain.invoke("cuando entregan el proyecto A?")
print(response)

El proyecto A, "Residencial Bosque Verde", se entregará en el segundo semestre del año 2026.


## RAG Graph

In [14]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [19]:
response = graph.invoke({"question": "cual es el proyecto mas barato y mas costoso? dame la diferencia de precios"})
print(response["answer"])

El proyecto más barato es el **Proyecto A: Residencial "Bosque Verde"** y el más costoso es el **Proyecto C: Residencial de lujo "Altos del Mar"**. La diferencia de precios entre ambos proyectos es de $300,000.
